## LoFTR stands for Detector-Free Local Feature Matching with Transformers. 

It is detailed described in [LoFTR: Detector-Free Local Feature Matching with Transformers](https://arxiv.org/pdf/2104.00680.pdf)(1). 

Source[1]: 
Novel method for local image feature matching. Instead of performing image feature detection, description, and matching sequentially, we propose to first establish pixel-wise dense matches at a coarse level and later refine the good matches at a fine level. In contrast
to dense methods that use a cost volume to search correspondences, we use self and cross attention layers in Transformer to obtain feature descriptors that are conditioned on both images. The global receptive field provided by Transformer enables our method to produce dense matches in low-texture areas, where feature detectors usually struggle to produce repeatable interest points. The experiments on indoor and outdoor datasets show that LoFTR outperforms state-of-the-art methods by a large margin. LoFTR also ranks first on two public benchmarks of visual localization among the published methods. Code is available at  project page: https://zju3dv.github.io/loftr/


<div align = "center"><img src="https://i.ibb.co/mygCR9n/LoFTR.png"/></div>


In this notebook I will show you how use LoFTR using Kornia for matching whales features. Kornia is a differentiable library that allows classical computer vision to be integrated into deep learning models.

It consists of a set of routines and differentiable modules to solve generic computer vision problems. At its core, the package uses PyTorch as its main backend both for efficiency and to take advantage of the reverse-mode auto-differentiation to define and compute the gradient of complex functions.

Why Kornia ?
With Kornia we fill the gap between classical and deep computer vision that implements standard and advanced vision algorithms for AI:

* Computer Vision: Kornia fills the gap between Classical and Deep computer Vision.
* Differentiable: We leverage the Computer Vision 2.0 paradigm.
* Open Source: Our libraries and initiatives are always according to the community needs.
* PyTorch: At our core we use PyTorch and its Autograd engine for its efficiency.

In [ ]:
%%capture

!pip install git+https://github.com/kornia/kornia
!pip install kornia_moons

In [ ]:
import cv2
import kornia as K
import kornia.feature as KF
from kornia.feature.loftr import LoFTR
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import glob
import random

from kornia_moons.feature import *
from PIL import Image

In [ ]:
def plot_images(ims):
    
    fig, axes = plt.subplots(3, 3, figsize=(20,20))
    
    for idx, img in enumerate(ims):
        i = idx % 3 
        j = idx // 3 
        image = Image.open(img)
        image = image.resize((300,300))
        axes[i, j].imshow(image)
        axes[i, j].set_title(img.split('/')[-1])

    plt.subplots_adjust(wspace=0, hspace=.2)
    plt.show()

In [ ]:
def load_torch_image(fname):
    img = K.image_to_tensor(cv2.imread(fname), False).float() /255.
    img = K.color.bgr_to_rgb(img)
    return img

In [ ]:
def match_and_draw(img_in1, img_in2):
    img1 = load_torch_image(img_in1)
    img2 = load_torch_image(img_in2)
    matcher = LoFTR(pretrained='outdoor')
    
    input_dict = {"image0": K.color.rgb_to_grayscale(img1), 
                  "image1": K.color.rgb_to_grayscale(img2)}
    
    with torch.no_grad():
        correspondences = matcher(input_dict)
    
    mkpts0 = correspondences['keypoints0'].cpu().numpy()
    mkpts1 = correspondences['keypoints1'].cpu().numpy()
    H, inliers = cv2.findFundamentalMat(mkpts0, mkpts1, cv2.USAC_MAGSAC, 0.5, 0.999, 100000)
    inliers = inliers > 0
    
    draw_LAF_matches(
    KF.laf_from_center_scale_ori(torch.from_numpy(mkpts0).view(1,-1, 2),
                                torch.ones(mkpts0.shape[0]).view(1,-1, 1, 1),
                                torch.ones(mkpts0.shape[0]).view(1,-1, 1)),

    KF.laf_from_center_scale_ori(torch.from_numpy(mkpts1).view(1,-1, 2),
                                torch.ones(mkpts1.shape[0]).view(1,-1, 1, 1),
                                torch.ones(mkpts1.shape[0]).view(1,-1, 1)),
    torch.arange(mkpts0.shape[0]).view(-1,1).repeat(1,2),
    K.tensor_to_image(img1),
    K.tensor_to_image(img2),
    inliers,
    draw_dict={'inlier_color': (0.2, 1, 0.2),
               'tentative_color': None, 
               'feature_color': (0.2, 0.5, 1), 'vertical': False})
    return correspondences

In [ ]:
def plot_matching(samples, files):
    for i in range(samples.shape[1]):
        image_1 = files[samples[0][i]]
        image_2 = files[samples[1][i]]
        print(f'Matching: {image_1} to {image_2}')
        correspondences = match_and_draw(image_1, image_2)

## SIMILARITY CHECK

In [ ]:
brandenburg_gate_path =  '../input/image-matching-challenge-2022/train/brandenburg_gate/images/'
brandenburg_gate_files = [file for file in glob.glob(f'{brandenburg_gate_path}*.jpg')]

plot_images(random.sample(brandenburg_gate_files, 9))

In [ ]:
random_samples = np.random.randint(len(brandenburg_gate_files), size=(2, 4))

plot_matching(random_samples, brandenburg_gate_files)

In [ ]:
sagrada_familia_path =  '../input/image-matching-challenge-2022/train/sagrada_familia/images/'
sagrada_familia_files = [file for file in glob.glob(f'{sagrada_familia_path}*.jpg')]

plot_images(random.sample(sagrada_familia_files, 9))

In [ ]:
random_samples = np.random.randint(len(sagrada_familia_files), size=(2, 4))

plot_matching(random_samples, sagrada_familia_files)

## DISSIMILARITY CHECK

In [ ]:
samples = np.array([[0],[1]])
files = [sagrada_familia_files[0], brandenburg_gate_files[0]]

plot_matching(samples, files)